In [1]:
import pandas as pd
from pathlib import Path

path = Path('C:/WIP/Projects/stregis_data/data')
file_paths = list(path.rglob('*.xlsx'))

rollup = pd.ExcelFile('C:\\WIP\\Projects\\stregis_data\\rollups_01.xlsx')
reports = rollup.sheet_names

series = pd.DataFrame()

for file in file_paths:
    temp = pd.read_excel(file, header=[1])
    series = pd.concat([series, temp])
    print(f'Processing file in path {file}')

series['Journal Date'] = pd.to_datetime(series['Journal Date'])
series['Year-Month'] = series['Journal Date'].dt.strftime('%Y-%m')

filtered_series = series[['Journal Date', 'Year-Month', 'DeptID', 'Dept Description' , 'Account', 'Descr', 'Base Currency', 'Base Amount']]
filtered_series_group = filtered_series.groupby(['Year-Month', 'DeptID', 'Dept Description', 'Account', 'Descr'])['Base Amount'].sum()

series_new = filtered_series_group.reset_index()

def generate_series(account_keys):
    results = []
    for key in account_keys:
        df_list = []
        
        for sheet in reports:
            series_temp = series_new[
                (series_new['DeptID'].isin(rollup.parse(sheet)['ID'].values.tolist())) &
                (series_new['Account'].astype(str).str.startswith(str(key)))
            ]
            
            if series_temp.empty:
                temp_df = pd.DataFrame(series['Year-Month'].unique(), columns=['Year-Month'])
                temp_df['Base Amount'] = 0
                grouped_df = temp_df.sort_values(by='Year-Month').reset_index(drop=True)
                df_list.append(grouped_df.rename(columns={'Year-Month': 'Date', 'Base Amount': sheet}))
            else:
                grouped_df = series_temp.groupby('Year-Month')['Base Amount'].sum().reset_index()
                grouped_df = grouped_df.sort_values(by='Year-Month')
                df_list.append(grouped_df.rename(columns={'Year-Month': 'Date', 'Base Amount': sheet}))

        combined_df = df_list[0]
        
        for df in df_list[1:]:
            combined_df = combined_df.merge(df, on='Date', how='outer')
        combined_df = combined_df.fillna(0)
        
        for col in combined_df.columns:
            if col != 'Date':
                combined_df[col] = combined_df[col] * (-1)
        results.append(combined_df)

    return pd.concat(results, axis=0).sort_values('Date').reset_index(drop=True)

########### Revenues, COGS / OPEX ###################

revenues = generate_series([3])
cogs_opex = generate_series([4]) + generate_series([5]) + generate_series([6]) + generate_series([7])

################ Base management fee ################

base_management_fee = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
base_management_fee_temp = series_new[series_new['Account'] == 706001].groupby(['Year-Month'])['Base Amount'].sum().reset_index()

base_management_fee = base_management_fee.join(base_management_fee_temp.set_index('Year-Month'), on = 'Year-Month')
base_management_fee = base_management_fee.sort_values(by = 'Year-Month', ascending = True)
base_management_fee = base_management_fee.rename(columns={'Year-Month': 'Date','Base Amount': 'Base management fee'})
base_management_fee = base_management_fee.fillna(0)
base_management_fee = base_management_fee.set_index('Date')
base_management_fee['Base management fee'] = base_management_fee['Base management fee'] * (-1)

################ Incentive fee ####################

incentive_fee = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
incentive_fee_temp = series_new[series_new['Account'] == 701125].groupby(['Year-Month'])['Base Amount'].sum().reset_index()

incentive_fee = incentive_fee.join(incentive_fee_temp.set_index('Year-Month'), on = 'Year-Month')
incentive_fee = incentive_fee.sort_values(by = 'Year-Month', ascending = True)
incentive_fee = incentive_fee.rename(columns={'Year-Month': 'Date','Base Amount': 'Incentive fee'})
incentive_fee = incentive_fee.fillna(0)
incentive_fee = incentive_fee.set_index('Date')
incentive_fee['Incentive fee'] = incentive_fee['Incentive fee'] * (-1)

################### FF&E reserve ####################

ffe_reserve = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
ffe_reserve_temp = series_new[series_new['Account'] == 701110].groupby(['Year-Month'])['Base Amount'].sum().reset_index()

ffe_reserve = ffe_reserve.join(ffe_reserve_temp.set_index('Year-Month'), on = 'Year-Month')
ffe_reserve = ffe_reserve.sort_values(by = 'Year-Month', ascending = True)
ffe_reserve = ffe_reserve.rename(columns={'Year-Month': 'Date','Base Amount': 'FFE Reserve'})
ffe_reserve = ffe_reserve.fillna(0)
ffe_reserve = ffe_reserve.set_index('Date')
ffe_reserve['FFE Reserve'] = ffe_reserve['FFE Reserve'] * (-1)


################### Property tax ####################

property_tax = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
property_tax_temp = series_new[series_new['Account'] == 720701].groupby(['Year-Month'])['Base Amount'].sum().reset_index()

property_tax = property_tax.join(property_tax_temp.set_index('Year-Month'), on = 'Year-Month')
property_tax = property_tax.sort_values(by = 'Year-Month', ascending = True)
property_tax = property_tax.rename(columns={'Year-Month': 'Date','Base Amount': 'Property tax'})
property_tax = property_tax.fillna(0)
property_tax = property_tax.set_index('Date')
property_tax['Property tax'] = property_tax['Property tax'] * (-1)

################### Insurance ####################

insurance_accounts = [730002, 730108]
insurance = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
insurance_temp = series_new[(series_new['Account'].isin(insurance_accounts) )].groupby(['Year-Month'])['Base Amount'].sum().reset_index()
insurance = insurance.join(insurance_temp.set_index('Year-Month'), on = 'Year-Month')
insurance = insurance.sort_values(by = 'Year-Month', ascending = True)
insurance = insurance.rename(columns={'Year-Month': 'Date','Base Amount': 'Insurance'})
insurance = insurance.fillna(0)
insurance = insurance.set_index('Date')
insurance['Insurance'] = insurance['Insurance'] * (-1)

################### Other non operating expenses ####################

other_nonop_expenses_accounts = [701130, 701502, 701503, 730107, 765257, 765259, 766931]
other_nonop_expenses = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
other_nonop_expenses_temp = series_new[(series_new['Account'].isin(other_nonop_expenses_accounts) )].groupby(['Year-Month'])['Base Amount'].sum().reset_index()
other_nonop_expenses = other_nonop_expenses.join(other_nonop_expenses_temp.set_index('Year-Month'), on = 'Year-Month')
other_nonop_expenses = other_nonop_expenses.sort_values(by = 'Year-Month', ascending = True)
other_nonop_expenses = other_nonop_expenses.rename(columns={'Year-Month': 'Date','Base Amount': 'Other Non-Op Expenses'})
other_nonop_expenses = other_nonop_expenses.fillna(0)
other_nonop_expenses = other_nonop_expenses.set_index('Date')
other_nonop_expenses['Other Non-Op Expenses'] = other_nonop_expenses['Other Non-Op Expenses'] * (-1)

################ Owner expenses ####################

owner_expenses_accounts = [700306, 700341, 765273, 766931]
owner_expenses = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
owner_expenses_temp = series_new[(series_new['Account'].isin(owner_expenses_accounts) )].groupby(['Year-Month'])['Base Amount'].sum().reset_index()
owner_expenses = owner_expenses.join(owner_expenses_temp.set_index('Year-Month'), on = 'Year-Month')
owner_expenses = owner_expenses.sort_values(by = 'Year-Month', ascending = True)
owner_expenses = owner_expenses.rename(columns={'Year-Month': 'Date','Base Amount': 'Owner expenses'})
owner_expenses = owner_expenses.fillna(0)
owner_expenses = owner_expenses.set_index('Date')
owner_expenses['Owner expenses'] = owner_expenses['Owner expenses'] * (-1)

################ Net to owner ####################

net_to_owner = pd.DataFrame(series['Year-Month'].unique()).rename(columns={0:'Year-Month'})
net_to_owner_temp = series_new[series_new['Account'] == 700304].groupby(['Year-Month'])['Base Amount'].sum().reset_index()

net_to_owner = net_to_owner.join(net_to_owner_temp.set_index('Year-Month'), on = 'Year-Month')
net_to_owner = net_to_owner.sort_values(by = 'Year-Month', ascending = True)
net_to_owner = net_to_owner.rename(columns={'Year-Month': 'Date','Base Amount': 'Net to owner'})
net_to_owner = net_to_owner.fillna(0)
net_to_owner = net_to_owner.set_index('Date')
net_to_owner['Net to owner'] = net_to_owner['Net to owner'] * (-1)

with pd.ExcelWriter('STRPM_datatape.xlsx', engine = 'openpyxl') as writer:
    series_new.to_excel(writer, sheet_name = 'STPRM_datatape', index = False)

with pd.ExcelWriter('exported_file.xlsx', engine = 'openpyxl') as writer:
    revenues.to_excel(writer, sheet_name = 'Revenues', index = False)
    cogs_opex.to_excel(writer, sheet_name = 'OPEX', index = False)
    base_management_fee.to_excel(writer, sheet_name = 'Base management fee', index = True)
    incentive_fee.to_excel(writer, sheet_name = 'Incentive fee', index = True)
    ffe_reserve.to_excel(writer, sheet_name = 'FF&E reserve', index = True)
    property_tax.to_excel(writer, sheet_name = 'Property tax', index = True)
    insurance.to_excel(writer, sheet_name = 'Insurance', index = True)
    other_nonop_expenses.to_excel(writer, sheet_name = 'Other Non-Op Expenses', index = True)
    owner_expenses.to_excel(writer, sheet_name = 'Owner expenses', index = True)
    net_to_owner.to_excel(writer, sheet_name = 'Net to owner', index = True)

Processing file in path C:\WIP\Projects\stregis_data\data\01 AUXILIAR OHVM ENERO 2022.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\01 AUXILIAR OHVM ENERO 2023.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\01 AUXILIAR OHVM ENERO 2024.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\02 AUXILIAR OHVM FEBRERO 2022.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\02 AUXILIAR OHVM FEBRERO 2023.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\02 AUXILIAR OHVM FEBRERO 2024.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\03 AUXILIAR OHVM MARZO 2022.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\03 AUXILIAR OHVM MARZO 2023.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\03 AUXILIAR OHVM MARZO 2024.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\04 AUXILIAR OHVM ABRIL 2022.xlsx
Processing file in path C:\WIP\Projects\stregis_data\data\04 AUXILIAR OHVM ABRIL 202